# Hydrogenation Case

<!-- <div><img src="cases/hydrogenation_process_scheme.jpg" width="600"/></div> -->
![](./cases/hydrogenation_process_scheme.png)

This case models a chemical process in which R-di-yne (RDY) undergoes hydrogenation to produce R-di-ethyl (RDEt). 
During the process, RDY and hydrogen gas are dissolved in a mixed liquid of water and an unknown solvent, with 
agitation applied to promote mass transfer. A cooling jacket is used to remove the heat generated by the exothermic 
hydrogenation reactions.  
The di-yne group of RDY is progressively hydrogenated by the catalyst (cat-H) through a sequence of intermediates — 
RDY → RYE → RYA → REA → RDEt. In addition, side reactions leading to dimer formation may occur during the process.

The reaction scheme is like:  
![](./cases/hydrogenation_reaction_scheme.png)

Reactions:
- H2 + cat2 > 2 cat-H
- 2 cat-H > H2 + cat2
- RDY + 2 cat-H > RYE + cat2
- RYE + 2 cat-H > RYA + cat2
- RYA + 2 cat-H > REA + cat2
- REA + 2 cat-H > RDEt + cat2
- RDEt + catX > catX-RDEt
- catX-RDEt > RDEt + catX
- RYE + catX-RDEt > dimer + catX
- REA + catX-RDEt > 2 H2 + dimer + catX

Mass Transfer:  
The gas-liquid and solid-liquid coefficients are correlated to agitation speed as $k_La = DN^\alpha$ and $k_Sa = DN^\alpha$, respectively

In this notebook, we go through model simulation and identify the rate-limiting step of the hydrogenation reaction.  
Then, sensitivity analysis is applied to identify parameters for model calibration.  
Finally, we run model calibration with different mass transport behaviours for model identification.

In [1]:
# Import required python libraries
import pandas as pd
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

from cases.hydrogenation_agitation import Hydrogenation


# Define phenomenon for the process to define the model
# RDY saturated concentration is fitted with RDEt concentration
# Supported phenomenon options:
# "Mass transport":
#       []
#       ["Gas-Liquid_Mass_Transfer"]
#       ["Solid-Liquid_Mass_Transfer"]
#       ["Gas-Liquid_Mass_Transfer", "Solid-Liquid_Mass_Transfer"],
# if "Gas-Liquid_Mass_Transfer" occurs, "Gas_Dissolution_Saturated_Concentration" can be "Henry's_Law" or "Constant" 
# if "Solid-Liquid_Mass_Transfer" occurs, "Solid_Dissolution_Saturated_Concentration" can be "Fitted" or "Constant" 
# RDY dissolution saturated concentration is found dependent on RDEt concentration, a relationship has been fitted and 
# included in the `hydrogenation.py` file for modelling
phenos = {
    "Mass accumulation":    "Batch",
    "Flow pattern":         "Well_Mixed",
    "Mass transport":       ["Gas-Liquid_Mass_Transfer", "Solid-Liquid_Mass_Transfer"],
    "param law":            {
        "Gas_Dissolution_Saturated_Concentration": "Henry's_Law",
        "Solid_Dissolution_Saturated_Concentration": "Fitted", 
    },
}

hydrogenation = Hydrogenation(phenos, random_seed=42)

### Model Simulation and Rate-limiting Step analysis
In this section, we run hydrogenation simulation with guessed kinetic parameters to visualise concentration profiles of reactants, intermediates, and products.  
Rate-limiting step is analysed by plotting intermediate mass profiles and rates of reaction and mass transport, which is further confirmed by sensitivity analysis.

In [2]:
# List operation parameters
# During experiments, masses of used water, solvent, and catalysts are fixed
for operation_param in hydrogenation.operation_params():
    print(operation_param)

({'param': 'Initial_Mass', 'stream': 'Batch_Feed', 'species': 'water'}, 0.2574)
({'param': 'Initial_Mass', 'stream': 'Batch_Feed', 'species': 'solvent'}, 0.194)
({'param': 'Initial_Mass', 'stream': 'Batch_Feed', 'species': 'cat2'}, 0.00235)
({'param': 'Initial_Mass', 'stream': 'Batch_Feed', 'species': 'catX'}, 0.00235)
({'param': 'Mass_Gas_Fraction', 'gas': 'Gas_Flow', 'species': 'H2'}, 1.0)
({'param': 'Initial_Mass_Solid', 'solid': 'Solid_Feedstock', 'species': 'RDY'}, None)
({'param': 'Temperature'}, None)
({'param': 'Pressure'}, None)
({'param': 'Agitation'}, None)
({'param': 'Batch_Time'}, None)


In [3]:
# Model simulation and concentration profile plot
operation_params = [
    ({"param": "Initial_Mass_Solid", "solid": "Solid_Feedstock", "species": "RDY"}, 0.10444), # kg
    ({"param": "Temperature"}, 20), # oC
    ({"param": "Pressure"},     4), # bar
    ({"param": "Agitation"}, 1200), # rpm
    ({"param": "Batch_Time"}, 309), # min
]
hydrogenation.plot_simulation_profiles(operation_params)

In [4]:
# Plot all reaction and mass transfer rates
# Rates of "H2 dissolution" and "H2 + cat2 <> 2 cat-H" are relatively fast
operation_params = [
    ({"param": "Initial_Mass_Solid", "solid": "Solid_Feedstock", "species": "RDY"}, 0.10444), # kg
    ({"param": "Temperature"}, 20), # oC
    ({"param": "Pressure"},     4), # bar
    ({"param": "Agitation"}, 1200), # rpm
    ({"param": "Batch_Time"}, 309), # min
]
steps = [
    "H2 + cat2 > 2 cat-H",
    "2 cat-H > H2 + cat2",
    "RDY + 2 cat-H > RYE + cat2",
    "RYE + 2 cat-H > RYA + cat2",
    "RYA + 2 cat-H > REA + cat2",
    "REA + 2 cat-H > RDEt + cat2",
    "RDEt + catX > catX-RDEt",
    "catX-RDEt > RDEt + catX",
    "RYE + catX-RDEt > dimer + catX",
    "REA + catX-RDEt > 2 H2 + dimer + catX",
    "H2 dissolution", 
    "RDY dissolution"
]
hydrogenation.plot_simulation_rates(steps, operation_params)

In [5]:
# Plot only rates of reactions from RDY to RDEt and RDY dissolution rate
# note that there's rate-limiting step in the series of hydrogenation reaction based on the overlapped rate curves
# RDY and RYE accumulate rapidly and then decay slowly
# if "RDY dissolution" is the rate-limiting step, there will be no RDY and RYA mass accumulation
# if "RYA + 2 cat-H > REA + cat2" is the rate-limiting step, RYA will accumulate which is not the case
# Therefore, "RDY + 2 cat-H > RYE + cat2" is the rate-limiting step, and RDY concentration rises close
# to the saturated concentration to slow down the solid RDY consumption rate
operation_params = [
    ({"param": "Initial_Mass_Solid", "solid": "Solid_Feedstock", "species": "RDY"}, 0.10444), # kg
    ({"param": "Temperature"}, 20), # oC
    ({"param": "Pressure"},     4), # bar
    ({"param": "Agitation"}, 1200), # rpm
    ({"param": "Batch_Time"}, 309), # min
]
steps = [
    "RDY + 2 cat-H > RYE + cat2",
    "RYE + 2 cat-H > RYA + cat2",
    "RYA + 2 cat-H > REA + cat2",
    "REA + 2 cat-H > RDEt + cat2",
    "RDY dissolution"
]
hydrogenation.plot_simulation_rates(steps, operation_params)

In [6]:
# Plot sensitivity analysis results
# We increase some parameters to 1.5 times of their values and visualise their influences on output
operation_params = [
    ({"param": "Initial_Mass_Solid", "solid": "Solid_Feedstock", "species": "RDY"}, 0.10444), # kg
    ({"param": "Temperature"}, 20), # oC
    ({"param": "Pressure"},     4), # bar
    ({"param": "Agitation"}, 1200), # rpm
    ({"param": "Batch_Time"}, 309), # min
]
varied_mechanistic_params = {
    "original": None,
    "H2 dissolution": {
        "param": "Gas-Liquid_Volumetric_Mass_Transfer_Coefficient_D", 
        "gas": "Gas_Flow", 
        "stream": "Batch_Feed", 
        "species": "H2"
    },
    "RDY dissolution": {
        "param": "Solid-Liquid_Volumetric_Mass_Transfer_Coefficient_D", 
        "solid": "Solid_Feedstock", 
        "stream": "Batch_Feed", 
        "species": "RDY"
    },
    "RDY hydrogenation": {
        "param": "Referenced_Reaction_Rate_Constant", 
        "stream": "Batch_Feed", 
        "reaction": "RDY + 2 cat-H > RYE + cat2"
    },
    "RYA hydrogenation": {
        "param": "Referenced_Reaction_Rate_Constant", 
        "stream": "Batch_Feed", 
        "reaction": "RYA + 2 cat-H > REA + cat2"
    },
    "Dimer generation1": {
        "param": "Referenced_Reaction_Rate_Constant", 
        "stream": "Batch_Feed", 
        "reaction": "RYE + catX-RDEt > dimer + catX"
    },
    "Dimer generation2": {
        "param": "Referenced_Reaction_Rate_Constant", 
        "stream": "Batch_Feed", 
        "reaction": "REA + catX-RDEt > 2 H2 + dimer + catX"
    },
}
hydrogenation.plot_sensitivity_analysis(varied_mechanistic_params, operation_params)

### Model calibration and identification
We first compare prediction vs. experiment data using parameters without calibration

In [7]:
# load experiment data and print
exp_dataset = pd.read_csv("./cases/hydrogenation_dataset.csv")
exp_dataset

,m_rdy,t_b,temp,pressure,agitation,m_rdet,m_dimer,h2_used,m_rdy_left
0,0.10444,311,10,4,1200,0.0662,0.00244,0.00339,0.03330
1,0.10444,309,20,4,1200,0.0838,0.00663,0.00445,0.01149
2,0.09060,135,25,12,1200,0.0862,0.00362,0.00454,0.00252
3,0.08520,352,25,4,1200,0.0739,0.00813,0.00410,0.00209
4,0.08770,352,5,12,1200,0.0766,0.00133,0.00381,0.01380
5,0.09120,234,20,10,600,0.0808,0.00407,0.00437,0.00330
6,0.06910,239,25,4,600,0.0683,0.00881,0.00350,0.00016


In [8]:
# plot parity plots
hydrogenation.plot_simulation_parity(exp_dataset)

#### Determining parameters for calibration
Recall reactions and mass transport phenomena
- H2 + cat2 > 2 cat-H
- 2 cat-H > H2 + cat2
- RDY + 2 cat-H > RYE + cat2
- RYE + 2 cat-H > RYA + cat2
- RYA + 2 cat-H > REA + cat2
- REA + 2 cat-H > RDEt + cat2
- RDEt + catX > catX-RDEt
- catX-RDEt > RDEt + catX
- RYE + catX-RDEt > dimer + catX
- REA + catX-RDEt > 2 H2 + dimer + catX
- hydrogen dissolution
- RDY dissolution

Measurements
- RDEt
- Dimer
- RDY undissolved
- H2 used

Parameters to be calibrated and their pristine values
- RDY hydrogenation reaction [RDY + 2 cat-H > RYE + cat2]  
    Referenced_Reaction_Rate_Constant: 3 mol/L s  
    Activation_Energy: 30 kJ/mol
- Dimer generation reaction [RYE + catX-RDEt > dimer + catX]  
    Referenced_Reaction_Rate_Constant: 1 mol/L s  
    Activation_Energy: 38 kJ/mol
- Dimer generation reaction [REA + catX-RDEt > 2 H2 + dimer + catX]  
    Referenced_Reaction_Rate_Constant: 1 mol/L s  
    Activation_Energy: 38 kJ/mol

In [9]:
# run model calibration with differential_evolution algorithm for complex reaction network model
cal_param_ranges = [
    (
        {"param": "Referenced_Reaction_Rate_Constant", 
         "stream": "Batch_Feed", 
         "reaction": "RDY + 2 cat-H > RYE + cat2"}, 
        (0, 5)
    ),
    (
        {"param": "Referenced_Reaction_Rate_Constant", 
         "stream": "Batch_Feed", 
         "reaction": "RYE + catX-RDEt > dimer + catX"}, 
        (0, 10)
    ),
    (
        {"param": "Referenced_Reaction_Rate_Constant", 
         "stream": "Batch_Feed", 
         "reaction": "REA + catX-RDEt > 2 H2 + dimer + catX"}, 
        (0, 10)
    ),
    (
        {"param": "Activation_Energy", 
         "stream": "Batch_Feed", 
         "reaction": "RDY + 2 cat-H > RYE + cat2"}, 
        (20, 80)
    ),
    (
        {"param": "Activation_Energy", 
         "stream": "Batch_Feed", 
         "reaction": "RYE + catX-RDEt > dimer + catX"}, 
        (20, 80)
    ),
    (
        {"param": "Activation_Energy", 
         "stream": "Batch_Feed", 
         "reaction": "REA + catX-RDEt > 2 H2 + dimer + catX"}, 
        (20, 80)
    ),
    (
        {"param": "Solid-Liquid_Volumetric_Mass_Transfer_Coefficient_D", 
         "solid": "Solid_Feedstock", 
         "stream": "Batch_Feed",
         "species": "RDY"}, 
        (0, 0.5)
    ),
    (
        {"param": "Gas-Liquid_Volumetric_Mass_Transfer_Coefficient_D", 
         "gas": "Gas_Flow", 
         "stream": "Batch_Feed",
         "species": "H2"}, 
        (0, 0.01)
    ),
]
cal_params = hydrogenation.calibrate(cal_param_ranges, exp_dataset)

differential_evolution step 1: f(x)= 0.00038440122435757314
differential_evolution step 2: f(x)= 0.00038440122435757314
differential_evolution step 3: f(x)= 0.00038440122435757314
differential_evolution step 4: f(x)= 0.00038440122435757314
differential_evolution step 5: f(x)= 0.00038440122435757314
differential_evolution step 6: f(x)= 0.0003529656086186397
differential_evolution step 7: f(x)= 0.0003529656086186397
differential_evolution step 8: f(x)= 0.0003357342109087516
differential_evolution step 9: f(x)= 0.0003357342109087516
differential_evolution step 10: f(x)= 0.0003179945213583795
differential_evolution step 11: f(x)= 0.0003179945213583795
differential_evolution step 12: f(x)= 0.0003098293209127628
differential_evolution step 13: f(x)= 0.0003098293209127628
differential_evolution step 14: f(x)= 0.00030314362864726784
differential_evolution step 15: f(x)= 0.00030314362864726784
differential_evolution step 16: f(x)= 0.00029931678732733857
differential_evolution step 17: f(x)= 0.0

In [10]:
# Parity plot after model calibratioin
cal_reaction_params = [(k, v) for k, v in cal_params if k in [p[0] for p in hydrogenation.reaction_params()]]
cal_transport_params = [(k, v) for k, v in cal_params if k in [p[0] for p in hydrogenation.transport_params()]]
hydrogenation.plot_simulation_parity(exp_dataset, reaction_params=cal_reaction_params, transport_params=cal_transport_params)

In [11]:
# plot RDEt mass profiles under varied mechanistic parameters
operation_params = [
    ({"param": "Initial_Mass_Solid", "solid": "Solid_Feedstock", "species": "RDY"}, 0.10444), # kg
    ({"param": "Temperature"}, 20), # oC
    ({"param": "Pressure"},     4), # bar
    ({"param": "Agitation"}, 1200), # rpm
    ({"param": "Batch_Time"}, 309), # min
]
varied_mechanistic_params = {
    "original": None,
    "H2 dissolution": {
        "param": "Gas-Liquid_Volumetric_Mass_Transfer_Coefficient_D", 
        "gas": "Gas_Flow", 
        "stream": "Batch_Feed", 
        "species": "H2"
    },
    "RDY dissolution": {
        "param": "Solid-Liquid_Volumetric_Mass_Transfer_Coefficient_D", 
        "solid": "Solid_Feedstock", 
        "stream": "Batch_Feed", 
        "species": "RDY"
    },
    "RDY hydrogenation": {
        "param": "Referenced_Reaction_Rate_Constant", 
        "stream": "Batch_Feed", 
        "reaction": "RDY + 2 cat-H > RYE + cat2"
    },
    "RYA hydrogenation": {
        "param": "Referenced_Reaction_Rate_Constant", 
        "stream": "Batch_Feed", 
        "reaction": "RYA + 2 cat-H > REA + cat2"
    },
    "Dimer generation1": {
        "param": "Referenced_Reaction_Rate_Constant", 
        "stream": "Batch_Feed", 
        "reaction": "RYE + catX-RDEt > dimer + catX"
    },
    "Dimer generation2": {
        "param": "Referenced_Reaction_Rate_Constant", 
        "stream": "Batch_Feed", 
        "reaction": "REA + catX-RDEt > 2 H2 + dimer + catX"
    },
}
hydrogenation.plot_sensitivity_analysis(varied_mechanistic_params, operation_params, cal_reaction_params, cal_transport_params)

#### Fitting model with constant RDY saturated concentration
identify how should we model the RDY saturated concentration by comparing calibration results of different models

In [12]:
# Try to identify the best fitted model with different structures based on different phenomenons
# Here we recreate the model by injecting different phenomenons
# The hypotheis of this model is that the solid dissolution saturated concention is constant
phenos = {
    "Mass accumulation":    "Batch",
    "Flow pattern":         "Well_Mixed",
    "Mass transport":       ["Gas-Liquid_Mass_Transfer", "Solid-Liquid_Mass_Transfer"],
    "param law":            {
        "Gas_Dissolution_Saturated_Concentration": "Henry's_Law",
        "Solid_Dissolution_Saturated_Concentration": "Constant",
    },
}

hydrogenation = Hydrogenation(phenos, random_seed=42)

In [13]:
# Run calibration again to calibrate the solid saturation concentration along with other parameters
cal_param_ranges = [
    (
        {"param": "Referenced_Reaction_Rate_Constant", 
         "stream": "Batch_Feed", 
         "reaction": "RDY + 2 cat-H > RYE + cat2"}, 
        (0, 5)
    ),
    (
        {"param": "Referenced_Reaction_Rate_Constant", 
         "stream": "Batch_Feed", 
         "reaction": "RYE + catX-RDEt > dimer + catX"}, 
        (0, 10)
    ),
    (
        {"param": "Referenced_Reaction_Rate_Constant", 
         "stream": "Batch_Feed", 
         "reaction": "REA + catX-RDEt > 2 H2 + dimer + catX"}, 
        (0, 10)
    ),
    (
        {"param": "Activation_Energy", 
         "stream": "Batch_Feed", 
         "reaction": "RDY + 2 cat-H > RYE + cat2"}, 
        (20, 80)
    ),
    (
        {"param": "Activation_Energy", 
         "stream": "Batch_Feed", 
         "reaction": "RYE + catX-RDEt > dimer + catX"}, 
        (20, 80)
    ),
    (
        {"param": "Activation_Energy", 
         "stream": "Batch_Feed", 
         "reaction": "REA + catX-RDEt > 2 H2 + dimer + catX"}, 
        (20, 80)
    ),
    (
        {"param": "Solid-Liquid_Volumetric_Mass_Transfer_Coefficient_D", 
         "solid": "Solid_Feedstock", 
         "stream": "Batch_Feed",
         "species": "RDY"}, 
        (0, 0.5)
    ),
    (
        {"param": "Gas-Liquid_Volumetric_Mass_Transfer_Coefficient_D", 
         "gas": "Gas_Flow", 
         "stream": "Batch_Feed",
         "species": "H2"}, 
        (0, 0.01)
    ),
    (
        {"param": "Constant_Solid_Saturated_Concentration", 
         "solid": "Solid_Feedstock", 
         "stream": "Batch_Feed",
         "species": "RDY"}, 
        (0, 0.003)
    )
]
cal_params = hydrogenation.calibrate(cal_param_ranges, exp_dataset)

differential_evolution step 1: f(x)= 0.0007047571087240911
differential_evolution step 2: f(x)= 0.0007047571087240911
differential_evolution step 3: f(x)= 0.0007047571087240911
differential_evolution step 4: f(x)= 0.0007047571087240911
differential_evolution step 5: f(x)= 0.0007047571087240911
differential_evolution step 6: f(x)= 0.0006698597298065127
differential_evolution step 7: f(x)= 0.0006698597298065127
differential_evolution step 8: f(x)= 0.0006698597298065127
differential_evolution step 9: f(x)= 0.000663333322023111
differential_evolution step 10: f(x)= 0.000663333322023111
differential_evolution step 11: f(x)= 0.000663333322023111
differential_evolution step 12: f(x)= 0.000663333322023111
differential_evolution step 13: f(x)= 0.000663333322023111
differential_evolution step 14: f(x)= 0.000663333322023111
differential_evolution step 15: f(x)= 0.0006492808218655062
differential_evolution step 16: f(x)= 0.0006492808218655062
differential_evolution step 17: f(x)= 0.000649280821865

In [14]:
# Plot the pred vs. exp result
# Model with constant solid saturation concentration has a larger error after fitting
# This suggests that RDY dissolution is more reasonable to be modelled using the fitted relationship S_RDY = func([RDEt])
cal_reaction_params = [(k, v) for k, v in cal_params if k in [p[0] for p in hydrogenation.reaction_params()]]
cal_transport_params = [(k, v) for k, v in cal_params if k in [p[0] for p in hydrogenation.transport_params()]]
hydrogenation.plot_simulation_parity(exp_dataset, reaction_params=cal_reaction_params, transport_params=cal_transport_params)